In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

REBUILD_DATA = False

class DogsVSCats():
    IMG_SIZE = 50
    CATS = 'PetImages/Cat'
    DOGS = 'PetImages/Dog'
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []
    catcount = 0
    dogcount = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1
                except Exception as e:
                    pass
                
        np.random.shuffle(self.training_data)
        np.save('training_data.npy', self.training_data)
        print('Cats:', self.catcount)
        print('Dogs:', self.dogcount)
        
if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()
    
training_data = np.load('training_data.npy', allow_pickle = True)
print(len(training_data))

24946


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
#         print(x)
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim = 1)
    
net = Net()
net = net.double()

torch.Size([128, 2, 2])


In [32]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr = 0.001)
loss_function = nn.MSELoss()

X = torch.tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0
y = torch.tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

2494


In [36]:
train_X = X[:-val_size].double()
train_y = y[:-val_size].double()

test_X = X[-val_size:].double()
test_y = y[-val_size:].double()

print(len(train_X))
print(len(test_X))

22452
2494


In [34]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]
        net.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
print(loss)

  0%|          | 0/225 [00:00<?, ?it/s]

torch.Size([128, 2, 2])


  0%|          | 1/225 [00:00<02:52,  1.30it/s]

torch.Size([128, 2, 2])


  1%|          | 2/225 [00:01<02:44,  1.35it/s]

...

 99%|█████████▉| 223/225 [02:37<00:01,  1.31it/s]

torch.Size([128, 2, 2])


100%|█████████▉| 224/225 [02:38<00:00,  1.19it/s]

torch.Size([128, 2, 2])


100%|██████████| 225/225 [02:38<00:00,  1.42it/s]


In [37]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]
        print(real_class, net_out)
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total += 1
        
print("Accuracy:", round(correct/total, 3))

  2%|▏         | 43/2494 [00:00<00:15, 155.34it/s]

torch.Size([128, 2, 2])
tensor(0) tensor([0.5313, 0.4687], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.5549, 0.4451], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.6367, 0.3633], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.7407, 0.2593], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.4685, 0.5315], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.7444, 0.2556], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.7938, 0.2062], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.6203, 0.3797], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.4070, 0.5930], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.4791, 0.5209], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.6327, 0.3673], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.5044, 0.4956], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor

  4%|▍         | 105/2494 [00:00<00:11, 207.77it/s]

torch.Size([128, 2, 2])
tensor(0) tensor([0.6182, 0.3818], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.6508, 0.3492], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.6515, 0.3485], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.7022, 0.2978], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.6539, 0.3461], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.5805, 0.4195], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.4786, 0.5214], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.6598, 0.3402], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.5308, 0.4692], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.6222, 0.3778], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.8378, 0.1622], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.6523, 0.3477], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor

  7%|▋         | 170/2494 [00:00<00:09, 254.32it/s]

...

torch.Size([128, 2, 2])
tensor(0) tensor([0.7907, 0.2093], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.5688, 0.4312], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.7601, 0.2399], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.4726, 0.5274], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.5114, 0.4886], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.5701, 0.4299], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.5008, 0.4992], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.4998, 0.5002], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.8447, 0.1553], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.4518, 0.5482], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.8276, 0.1724], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.6844, 0.3156], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor

 96%|█████████▌| 2400/2494 [00:07<00:00, 310.46it/s]

tensor(1) tensor([0.1693, 0.8307], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.6124, 0.3876], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.6269, 0.3731], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.3216, 0.6784], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.2210, 0.7790], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.6760, 0.3240], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.2998, 0.7002], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.5534, 0.4466], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.7185, 0.2815], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.8457, 0.1543], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.4534, 0.5466], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.7397, 0.2603], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.2743, 0.7257], dtype

100%|██████████| 2494/2494 [00:08<00:00, 308.12it/s]

torch.Size([128, 2, 2])
tensor(0) tensor([0.7234, 0.2766], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.7000, 0.3000], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.4956, 0.5044], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.7915, 0.2085], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.3284, 0.6716], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.8153, 0.1847], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.4021, 0.5979], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.6743, 0.3257], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.6342, 0.3658], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor([0.5811, 0.4189], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.3797, 0.6203], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(1) tensor([0.4962, 0.5038], dtype=torch.float64)
torch.Size([128, 2, 2])
tensor(0) tensor